In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
# from linearmodels.panel import PanelOLS

In [40]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

## 2020 treatment vs control

In [52]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

wf["temp2"] = wf["temp"] ** 2
wf["l_aqi"] = np.log(1 + wf["aqi"])
wf["l_pm"] = np.log(1 + wf["pm"])
wf2020 = wf[(wf["daynum"] >= 8401) & (wf["daynum"]<= 8461)].dropna(
    subset = ['aqi', 'pm']
)
wf2020['cities'] = wf2020['city_code'].astype('category')
wf2020['days'] = wf2020['daynum'].astype('category')
wf2020 = pd.get_dummies(wf2020, drop_first=True)

In [53]:
fixed = ['treat']
for col in wf2020.columns:
    if 'cities' in col or 'days' in col:
        fixed.append(col)
        
weather = ['prec', 'snow', 'temp', 'temp2']

In [55]:
out = ["aqi", "l_aqi", "pm", "l_pm"]

for Yname in out:
    Y = wf2020[Yname]
    X = wf2020[fixed + weather]
    lr = LinearRegression()
    lr.fit(X, Y)
    print(lr.coef_[0])

-19.844075932282255
-0.1741297114238865
-14.069969671861704
-0.1678396393548723


## Spring festival 2020 vs 2019

In [75]:
wf_spring = wf[
    ((wf["daynum"] >= 8401) & (wf["daynum"]<= 8461)) |
    ((wf["daynum"] >= 8047) & (wf["daynum"]<= 8107))].dropna(
    subset = ['aqi', 'pm']
)
wf_spring["daynum_i"] = wf_spring.apply(lambda x : x["daynum"] + 354 if x["year"] == 2019 else x["daynum"], axis=1)
wf_spring["treat_SF1"] = ((wf_spring["daynum_i"] >= 8425) & (wf_spring["year"] == 2020)).astype("int")

wf_spring['cities'] = wf_spring['city_code'].astype('category')
wf_spring['days_i'] = wf_spring['daynum_i'].astype('category')
wf_spring['year_ind'] = wf_spring['year'].astype('category')
wf_spring['t_group'] = pd.notna(wf_spring["t_asign"])
wf_spring = pd.get_dummies(wf_spring, drop_first=True)


fixed = ['treat_SF1']
for col in wf_spring.columns:
    if 'cities' in col or 'days_i' in col or 'year_ind' in col:
        fixed.append(col)

In [76]:
for Yname in out:
    Y = wf_spring[wf_spring["t_group"] == 0][Yname]
    X = wf_spring[wf_spring["t_group"] == 0][fixed + weather]
    lr = LinearRegression()
    lr.fit(X, Y)
    print(lr.coef_[0])

-6.369863682777258
-0.0484466255348826
-7.090465831937396
-0.07072380133669583
